In [1]:
import os

if os.path.basename(os.getcwd()) == 'notebooks':
    os.chdir('..')
    
os.getcwd()

'/home/zack/personal/prediction_market_tools'

In [2]:
import pandas as pd

In [3]:
p = pd.read_csv('data/raw/predsnprices.csv')

In [4]:
from src import expected_value

In [5]:
p, latest_p = expected_value.prep_prices_csv(p)

/home/zack/personal/prediction_market_tools/src/expected_value.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latest_p["buyYes"] = latest_p.myBet > latest_p.yesPrice
/home/zack/miniconda3/envs/preds/lib/python3.9/site-packages/pandas/core/frame.py:3636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]
/home/zack/personal/prediction_market_tools/src/expected_value.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [6]:
p

,symbol,date,yesPrice,noPrice,myBet,purchase,qty
0,LCASE-001,2021-09-02,0.58,0.44,0.72,NaN,NaN
1,LCASE-001,2021-09-09,0.19,0.82,0.35,NaN,NaN
2,VOHC-001,2021-09-09,0.35,0.66,0.25,NaN,NaN
3,NYSCHL-001,2021-09-09,0.84,0.18,0.90,NaN,NaN
4,WKCASE-004,2021-09-13,0.57,0.45,0.63,NaN,NaN
5,LCASE-001,2021-09-14,0.11,0.91,0.15,NaN,NaN
6,VOHC-001,2021-09-14,0.36,0.66,0.25,n,98.0
7,NYSCHL-001,2021-09-14,0.84,0.17,0.90,y,125.0
8,WKCASE-004,2021-09-14,0.74,0.29,NaN,NaN,NaN
9,CASE-038,2021-09-14,0.07,0.95,0.05,NaN,NaN


In [7]:
latest_p

,symbol,date,yesPrice,noPrice,myBet,purchase,qty,buyYes,mktp,myp,mktq,myq,expected_value
6,VOHC-001,2021-09-14,0.36,0.66,0.25,n,98.0,False,0.66,0.75,0.34,0.25,0.09
7,NYSCHL-001,2021-09-14,0.84,0.17,0.90,y,125.0,True,0.84,0.90,0.16,0.10,0.06
9,CASE-038,2021-09-14,0.07,0.95,0.05,NaN,NaN,False,0.95,0.95,0.05,0.05,0.00
13,LCASE-001,2021-09-15,0.14,0.88,0.25,NaN,NaN,True,0.14,0.25,0.86,0.75,0.11
14,CASE-039,2021-09-15,0.06,NaN,0.05,NaN,NaN,False,NaN,0.95,NaN,0.05,NaN
16,CASE-040,2021-09-16,0.98,0.03,1.00,NaN,NaN,True,0.98,1.00,0.02,0.00,0.02
18,WKCASE-004,2021-09-16,0.99,0.02,1.00,NaN,NaN,True,0.99,1.00,0.01,0.00,0.01


In [8]:
from src import kalshi

In [9]:
k = kalshi.Kalshi()

In [10]:
k.login()

In [11]:
positions = k.get_positions()
positions

,user_id,market_id,position,position_cost,resting_orders_count,total_cost,realized_pnl,fees_paid,volume,final_position,final_position_cost
0,46ef47c2-585e-49e6-a0b2-d1fdd1f18ff4,2f485e17-27ac-4337-ad04-d63a9fc578ad,-98,6468,0,6468,0,154,98,0,0
1,46ef47c2-585e-49e6-a0b2-d1fdd1f18ff4,c549ccac-ad50-4f41-a1d1-771745ee97e5,125,10500,0,10500,0,119,125,0,0
2,46ef47c2-585e-49e6-a0b2-d1fdd1f18ff4,b9689b80-9810-4bbd-b48a-8c637be5f056,5,465,0,465,0,3,5,0,0


In [12]:
markets = k.select_markets(positions.market_id)
markets

,id,title,category,tags,ranged_group_name,image_url,min_tick_size,settle_details,settlement_sources,ticker_name,...,yes_bid,yes_ask,last_price,previous_yes_bid,previous_yes_ask,previous_price,volume,recent_volume,open_interest,result
0,2f485e17-27ac-4337-ad04-d63a9fc578ad,Will the CDC identify a variant of high conseq...,COVID-19,"[COVID-19, Variant]",The CDC’s SARS-CoV-2 Variant Classifications.,https://kalshi-public-images.s3.us-east-2.amaz...,$0.01,If the CDC identifies a variant of high conseq...,[{'url': 'https://www.cdc.gov/coronavirus/2019...,VOHC-001,...,32,36,34,33,36,34,2402,0,2060,
1,c549ccac-ad50-4f41-a1d1-771745ee97e5,Will New York public schools remain fully open...,COVID-19,"[New York, Education]",The reopening plans of the New York City publi...,https://kalshi-public-images.s3.us-east-2.amaz...,$0.01,If New York City public schools remain fully o...,"[{'url': 'https://www.schools.nyc.gov/', 'name...",NYSCHL-001,...,85,86,85,85,86,85,4742,190,2846,
2,b9689b80-9810-4bbd-b48a-8c637be5f056,"Will there be over 140,000 new COVID-19 cases ...",COVID-19,"[COVID, Cases]",The number of new cases of COVID-19 published ...,https://kalshi-public-images.s3.us-east-2.amaz...,$0.01,If the number of new cases of COVID-19 reporte...,[{'url': 'https://covid.cdc.gov/covid-data-tra...,CASE-040,...,99,100,98,97,99,98,5748,3604,1574,


In [13]:
positions = positions.merge(markets[['id', 'ticker_name', 'last_price']], how = 'left', left_on='market_id', right_on='id')
positions['mrkt_prob'] = positions.last_price.where(positions.position > 0, 100 - positions.last_price) / 100
positions['qty'] = positions.position.where(positions.position > 0, - positions.position)
positions.position_cost = positions.position_cost / 100 
positions = positions.merge(latest_p[['symbol','myBet']], how='left', left_on='ticker_name', right_on='symbol')
positions['myBet'] = positions.myBet.where(positions.position > 0, 1 - positions.myBet)
positions

,user_id,market_id,position,position_cost,resting_orders_count,total_cost,realized_pnl,fees_paid,volume,final_position,final_position_cost,id,ticker_name,last_price,mrkt_prob,qty,symbol,myBet
0,46ef47c2-585e-49e6-a0b2-d1fdd1f18ff4,2f485e17-27ac-4337-ad04-d63a9fc578ad,-98,64.68,0,6468,0,154,98,0,0,2f485e17-27ac-4337-ad04-d63a9fc578ad,VOHC-001,34,0.66,98,VOHC-001,0.75
1,46ef47c2-585e-49e6-a0b2-d1fdd1f18ff4,c549ccac-ad50-4f41-a1d1-771745ee97e5,125,105.00,0,10500,0,119,125,0,0,c549ccac-ad50-4f41-a1d1-771745ee97e5,NYSCHL-001,85,0.85,125,NYSCHL-001,0.90
2,46ef47c2-585e-49e6-a0b2-d1fdd1f18ff4,b9689b80-9810-4bbd-b48a-8c637be5f056,5,4.65,0,465,0,3,5,0,0,b9689b80-9810-4bbd-b48a-8c637be5f056,CASE-040,98,0.98,5,CASE-040,1.00


In [14]:
exp, situations = expected_value.compute_wager_expectations(positions.mrkt_prob, positions.position_cost, positions.qty)

# Market Expectation

In [15]:
exp

1.4999999999999964

In [16]:
situations

,hits,hit_percent,hits_p,hits_exp
0,0,0.000000,0.00102,-174.330000
1,1,0.333333,0.05774,-154.128407
2,2,0.666667,0.39146,-48.356158
3,3,1.000000,0.54978,53.670000


# My Expectation

In [17]:
exp, situations = expected_value.compute_wager_expectations(positions.myBet, positions.position_cost, positions.qty)

/home/zack/personal/prediction_market_tools/src/expected_value.py:51: RuntimeWarning: invalid value encountered in true_divide
  sit_outcome_ps = outcome_ps / situation_p


In [18]:
exp

16.67

In [19]:
situations

,hits,hit_percent,hits_p,hits_exp
0,0,0.000000,0.000,NaN
1,1,0.333333,0.025,-169.33
2,2,0.666667,0.300,-51.08
3,3,1.000000,0.675,53.67
